# 1. 데이터 준비 및 인덱싱 (Data Setup & Indexing)
- KomuRetrieval 전체 데이터셋 다운로드
- Kiwi 형태소 분석기를 이용한 전처리
- Inverted Index 구축 및 SQLite 저장

In [3]:
import os
import sqlite3
import json
import pickle
from collections import defaultdict, Counter
from typing import Dict, List
from tqdm.notebook import tqdm  # 주피터용 tqdm
from kiwipiepy import Kiwi
from datasets import load_dataset
from pathlib import Path

# 경로 설정
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
DB_DIR = PROJECT_ROOT / 'database'
DB_PATH = DB_DIR / 'inverted_index.db'

# 디렉토리 생성
for d in [DATA_DIR, DB_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"프로젝트 루트: {PROJECT_ROOT}")
print(f"데이터 저장소: {DATA_DIR}")
print(f"DB 저장소: {DB_PATH}")

프로젝트 루트: C:\Users\cse\Desktop\xeoxaxeo\NLP
데이터 저장소: C:\Users\cse\Desktop\xeoxaxeo\NLP\data
DB 저장소: C:\Users\cse\Desktop\xeoxaxeo\NLP\database\inverted_index.db


## 1.1 데이터셋 로드 (전체 데이터)
HuggingFace에서 데이터를 다운로드하고 pickle로 저장.

In [4]:
def load_and_save_dataset():
    print("HuggingFace 데이터셋 다운로드")
    queries = load_dataset("junyoungson/KomuRetrieval", "queries", split="queries")
    corpus = load_dataset("junyoungson/KomuRetrieval", "corpus", split="corpus")
    qrels = load_dataset("junyoungson/KomuRetrieval", split="test")

    # 리스트 변환
    print("데이터 리스트 변환")
    full_corpus = [item for item in tqdm(corpus, desc="Corpus 변환")]
    full_queries = [item for item in tqdm(queries, desc="Queries 변환")]
    full_qrels = [item for item in tqdm(qrels, desc="Qrels 변환")]

    # 앞에서부터 10,000개만 샘플링
    SAMPLE_SIZE = 10000
    full_corpus = full_corpus[:SAMPLE_SIZE]

    # 저장
    with open(DATA_DIR / 'corpus.pkl', 'wb') as f:
        pickle.dump(full_corpus, f)
    with open(DATA_DIR / 'queries.pkl', 'wb') as f:
        pickle.dump(full_queries, f)
    with open(DATA_DIR / 'qrels.pkl', 'wb') as f:
        pickle.dump(full_qrels, f)

    print(f"\nCorpus: {len(full_corpus):,}개 샘플링됨")
    return full_corpus

# 데이터가 없으면 다운로드, 있으면 로드
if not (DATA_DIR / 'corpus.pkl').exists():
    corpus = load_and_save_dataset()
else:
    print("이미 데이터가 존재합니다 -> 로드")
    with open(DATA_DIR / 'corpus.pkl', 'rb') as f:
        corpus = pickle.load(f)
    print(f"Corpus 로드 완료: {len(corpus):,}개")

# 이미 불러온 데이터에서 자르기
if len(corpus) > 10000:
    print(f"\n기존 데이터에서 상위 10,000개만 남기고 자르기")
    corpus = corpus[:10000]
    print(f"-> 적용 완료: {len(corpus):,}개")

이미 데이터가 존재합니다 -> 로드
Corpus 로드 완료: 50,222개

기존 데이터에서 상위 10,000개만 남기고 자르기
-> 적용 완료: 10,000개


## 1.2 형태소 분석 및 토큰화
Kiwi를 사용하여 명사, 동사, 형용사 등을 추출.

In [5]:
kiwi = Kiwi(num_workers=-1)

def tokenize(text: str) -> List[str]:
    if not text or not text.strip():
        return []

    clean_text = text.replace('\x00', '')

    try:
        tokens = kiwi.tokenize(clean_text)
        useful_tags = ['NNG', 'NNP', 'VV', 'VA', 'MAG']
        return [t.form for t in tokens if t.tag in useful_tags and len(t.form) > 1]
    except Exception:
        return []

# 테스트 실행
print(tokenize("이화여자대학교는 스크랜튼 여사가 1886년에 설립하였다."))

['이화여자대학교', '스크랜튼 여사', '설립']


## 1.3 Inverted Index 구축 및 DB 저장
시간이 다소 소요될 수 있음.

In [6]:
def build_index_db(corpus):
    conn = sqlite3.connect(str(DB_PATH))
    cursor = conn.cursor()

    # 기존 테이블 초기화
    cursor.execute("DROP TABLE IF EXISTS documents")
    cursor.execute("DROP TABLE IF EXISTS inverted_index")
    cursor.execute("DROP TABLE IF EXISTS statistics")
    cursor.execute("DROP TABLE IF EXISTS term_stats")

    # 테이블 생성
    cursor.execute("CREATE TABLE documents (doc_id TEXT PRIMARY KEY, title TEXT, length INTEGER, tokens TEXT)")
    cursor.execute("CREATE TABLE inverted_index (term TEXT, doc_id TEXT, tf INTEGER, PRIMARY KEY (term, doc_id))")
    cursor.execute("CREATE TABLE statistics (key TEXT PRIMARY KEY, value REAL)")
    cursor.execute("CREATE TABLE term_stats (term TEXT PRIMARY KEY, df INTEGER)")

    # 변수 초기화
    inverted_index = defaultdict(lambda: defaultdict(int))
    doc_lengths = {}

    # 문서 토큰화 및 저장
    doc_data = []
    for doc in tqdm(corpus, desc="Tokenizing"):
        doc_id = doc['_id']
        full_text = f"{doc['title']} {doc['text']}"
        tokens = tokenize(full_text)

        doc_lengths[doc_id] = len(tokens)
        doc_data.append((doc_id, doc['title'], len(tokens), json.dumps(tokens, ensure_ascii=False)))

        # 메모리 내 인덱스 구축
        counts = Counter(tokens)
        for term, tf in counts.items():
            inverted_index[term][doc_id] = tf

        # 배치 커밋 (메모리 관리)
        if len(doc_data) >= 1000:
            cursor.executemany("INSERT INTO documents VALUES (?, ?, ?, ?)", doc_data)
            conn.commit()
            doc_data = []

    if doc_data:
        cursor.executemany("INSERT INTO documents VALUES (?, ?, ?, ?)", doc_data)
        conn.commit()

    # Inverted Index 저장
    index_data = []
    term_stats_data = []

    for term, postings in tqdm(inverted_index.items(), desc="Saving Index"):
        df = len(postings)
        term_stats_data.append((term, df))

        for doc_id, tf in postings.items():
            index_data.append((term, doc_id, tf))

        if len(index_data) >= 10000:
            cursor.executemany("INSERT INTO inverted_index VALUES (?, ?, ?)", index_data)
            conn.commit()
            index_data = []

    if index_data:
        cursor.executemany("INSERT INTO inverted_index VALUES (?, ?, ?)", index_data)

    cursor.executemany("INSERT INTO term_stats VALUES (?, ?)", term_stats_data)

    # 통계 저장
    N = len(corpus)
    avgdl = sum(doc_lengths.values()) / N if N > 0 else 0
    cursor.executemany("INSERT INTO statistics VALUES (?, ?)", [
        ('N', N), ('avgdl', avgdl), ('total_terms', len(inverted_index))
    ])

    # 인덱싱
    cursor.execute("CREATE INDEX idx_term ON inverted_index(term)")
    cursor.execute("CREATE INDEX idx_doc_id ON inverted_index(doc_id)")

    conn.commit()
    conn.close()
    print("DB 구축 완료")

# 인덱싱 실행
build_index_db(corpus)

Tokenizing:   0%|          | 0/10000 [00:00<?, ?it/s]

Saving Index:   0%|          | 0/207107 [00:00<?, ?it/s]

DB 구축 완료
